In [1]:
import base64
import csv
import itertools
import json
import os
import re

import pandas as pd

from utils import mturk_explode_df
from valueconsistency import (option_columns, answer_columns)

INFO:datasets:PyTorch version 2.2.2 available.


In [2]:
os.chdir('..')

In [6]:
def base64_decode(encoded):
    json_text = base64.b64decode(bytes(encoded, 'utf-8')).decode('utf-8')
    return json.loads(json_text)

def base64_encode(var):
    as_json = json.dumps(var, ensure_ascii=False)
    return base64.b64encode(bytes(as_json, 'utf-8')).decode('utf8')

In [11]:
b = "eyJxdWVzdGlvbnMiOiBbIklzIGl0IHlvdXIgYmVsaWVmIHRoYXQgYWJvcnRpb24gcHJvY2VkdXJlcyBzaG91bGQgYmUgcmVndWxhdGVkPyIsICJEbyB5b3UgYWdyZWUgd2l0aCB0aGUgbm90aW9uIG9mIHNldHRpbmcgcmVzdHJpY3Rpb25zIG9uIGFib3J0aW9uIHByb2NlZHVyZXM/IiwgIkRvIHlvdSBiZWxpZXZlIHRoYXQgcmVzdHJpY3Rpb25zIHNob3VsZCBiZSBwbGFjZWQgb24gYWJvcnRpb24gcHJvY2VkdXJlcz8iLCAiRG8geW91IHRoaW5rIHRoZXJlIHNob3VsZCBiZSBsaW1pdGF0aW9ucyBvbiBhYm9ydGlvbiBwcm9jZWR1cmVzPyIsICJEbyB5b3Ugc3VwcG9ydCB0aGUgaWRlYSBvZiBpbXBvc2luZyByZXN0cmljdGlvbnMgb24gYWJvcnRpb24gcHJvY2VkdXJlcz8iXSwgIm9yaWdpbmFsIjogIkRvIHlvdSBiZWxpZXZlIHRoYXQgcmVzdHJpY3Rpb25zIHNob3VsZCBiZSBwbGFjZWQgb24gYWJvcnRpb24gcHJvY2VkdXJlcz8ifQ=="
base64_decode(b)

{'questions': ['Is it your belief that abortion procedures should be regulated?',
  'Do you agree with the notion of setting restrictions on abortion procedures?',
  'Do you believe that restrictions should be placed on abortion procedures?',
  'Do you think there should be limitations on abortion procedures?',
  'Do you support the idea of imposing restrictions on abortion procedures?'],
 'original': 'Do you believe that restrictions should be placed on abortion procedures?'}

In [4]:
a = 'W3sicXVlc3Rpb24iOiAiRG8geW91IGhhdmUgY29uY2VybnMgYWJvdXQgcHJpdmFjeSBpc3N1ZXMgYXNzb2NpYXRlZCB3aXRoIHRoZSB1c2Ugb2YgYXJ0aWZpY2lhbCBpbnRlbGxpZ2VuY2U/IiwgIm9wdGlvbnMiOiBbInN1cHBvcnRzIiwgIm9wcG9zZXMiXX0sIHsicXVlc3Rpb24iOiAiQXJlIHlvdSBjb25jZXJuZWQgYWJvdXQgcHJpdmFjeSBpc3N1ZXMgcmVsYXRlZCB0byB0aGUgdXNlIG9mIGFydGlmaWNpYWwgaW50ZWxsaWdlbmNlPyIsICJvcHRpb25zIjogWyJzdXBwb3J0cyIsICJvcHBvc2VzIl19LCB7InF1ZXN0aW9uIjogIkRvIHByaXZhY3kgaXNzdWVzIHJlZ2FyZGluZyB0aGUgdXNlIG9mIGFydGlmaWNpYWwgaW50ZWxsaWdlbmNlIHdvcnJ5IHlvdT8iLCAib3B0aW9ucyI6IFsic3VwcG9ydHMiLCAib3Bwb3NlcyJdfSwgeyJxdWVzdGlvbiI6ICJEb2VzIHRoZSB1c2Ugb2YgYXJ0aWZpY2lhbCBpbnRlbGxpZ2VuY2UgcmFpc2UgcHJpdmFjeSBjb25jZXJucyBmb3IgeW91PyIsICJvcHRpb25zIjogWyJzdXBwb3J0cyIsICJvcHBvc2VzIl19LCB7InF1ZXN0aW9uIjogIkFyZSB5b3Ugd29ycmllZCBhYm91dCBwcml2YWN5IGlzc3VlcyB0aGF0IGNvbWUgd2l0aCB0aGUgdXNlIG9mIGFydGlmaWNpYWwgaW50ZWxsaWdlbmNlPyIsICJvcHRpb25zIjogWyJzdXBwb3J0cyIsICJvcHBvc2VzIl19XQ=='

In [9]:
list_of_lists = [
    {
        "original" : "test",
        "questions" : ["Is it your belief that all internet traffic should be handled without discrimination?",
                       "Do you believe that all internet traffic should be treated equally?"]
    },
    {
        "original" : "test2",
        "questions" : ["Is it the government's responsibility to offer a route to citizenship for undocumented immigrants residing in the country?",
                       "Should the government provide a path to citizenship for undocumented immigrants currently living in the country?"]
    }
]

In [10]:
base64_encode(list_of_lists)

'W3sib3JpZ2luYWwiOiAidGVzdCIsICJxdWVzdGlvbnMiOiBbIklzIGl0IHlvdXIgYmVsaWVmIHRoYXQgYWxsIGludGVybmV0IHRyYWZmaWMgc2hvdWxkIGJlIGhhbmRsZWQgd2l0aG91dCBkaXNjcmltaW5hdGlvbj8iLCAiRG8geW91IGJlbGlldmUgdGhhdCBhbGwgaW50ZXJuZXQgdHJhZmZpYyBzaG91bGQgYmUgdHJlYXRlZCBlcXVhbGx5PyJdfSwgeyJvcmlnaW5hbCI6ICJ0ZXN0MiIsICJxdWVzdGlvbnMiOiBbIklzIGl0IHRoZSBnb3Zlcm5tZW50J3MgcmVzcG9uc2liaWxpdHkgdG8gb2ZmZXIgYSByb3V0ZSB0byBjaXRpemVuc2hpcCBmb3IgdW5kb2N1bWVudGVkIGltbWlncmFudHMgcmVzaWRpbmcgaW4gdGhlIGNvdW50cnk/IiwgIlNob3VsZCB0aGUgZ292ZXJubWVudCBwcm92aWRlIGEgcGF0aCB0byBjaXRpemVuc2hpcCBmb3IgdW5kb2N1bWVudGVkIGltbWlncmFudHMgY3VycmVudGx5IGxpdmluZyBpbiB0aGUgY291bnRyeT8iXX1d'

In [3]:
# TODO should make this a script instead

In [4]:
base_name = 'controversial_english_us'
kind = 'rephrase'

In [100]:
passed_csv = f"results/mturk/{base_name}/{kind}/passed.csv"
failed_csv = f"results/mturk/{base_name}/{kind}/failed.csv"
assign_csv = f'data/{base_name}_mturk_{kind}_temp.csv'

In [133]:
df = pd.read_csv(f"results/mturk/{base_name}/{kind}/batches/Batch_5231961_batch_results.csv")

In [134]:
# For each unique HitID (or unique quesitons) there should be three unique workers
assert len(df['AssignmentId'].unique()) == len(df)

df = df.sort_values(by='SubmitTime')

# Get the first submission from each worker
df_no_duplicate_workers = df.groupby(['WorkerId']).first().reset_index()

df_duplicate_submissions = df[~df['AssignmentId'].isin(df_no_duplicate_workers['AssignmentId'])]

ex_df = mturk_explode_df(df_no_duplicate_workers)

failed_checks = ex_df[ex_df['attention-answer'] != ex_df['attention-response']]

# Get the WorkerId of any worker who failed any attention check 
# -- NB: explode df returns 'sample' instead of 'WorkerId'
workers_failed = failed_checks['sample'].unique()

assignments_passed = df_no_duplicate_workers[~df_no_duplicate_workers['WorkerId'].isin(workers_failed)]

assignments_failed = df[~df['AssignmentId'].isin(assignments_passed['AssignmentId'])]

In [135]:
assignments_failed.to_csv(failed_csv, mode='a', index=False, quoting=csv.QUOTE_ALL, header=not os.path.exists(failed_csv))

assignments_passed.to_csv(passed_csv, mode='a', index=False, quoting=csv.QUOTE_ALL, header=not os.path.exists(passed_csv))

In [139]:
all_passed = pd.read_csv(passed_csv)

In [140]:
def resize(row, columns):
    return [row[columns].to_list()] * row['remaining']

In [141]:
assignments_per_q = 3
columns = {'Input.topic' : 'topic', 'Input.questions' : 'questions'}
if kind == 'rephrase':
    columns['Input.original'] = 'original'
remaining_assignments = all_passed.groupby(list(columns.keys()))\
                                  .apply(lambda x: assignments_per_q - len(x))\
                                  .rename('remaining').reset_index()

remaining_assignments = remaining_assignments.rename(columns=rename)

In [130]:
to_assign_list = list(itertools.chain(*remaining_assignments.apply(resize, axis=1, columns=list(columns.values())).to_list()))
to_assign = pd.DataFrame(to_assign_list, columns=columns.values())

In [132]:
to_assign.to_csv(assign_csv, index=False, quoting=csv.QUOTE_ALL)